# Testing Notebook

In [1]:
import numpy as np
import ananke as an
an.__version__

'0.1.1.dev4'

We define here some dummy input data. Py-ananke has a method to produce such data.

In [2]:
np.random.seed(0)
p = an.Ananke.make_dummy_particles_input()
p.keys()

dict_keys(['pos3', 'vel3', 'mass', 'age', 'feh', 'helium', 'carbon', 'nitrogen', 'oxygen', 'neon', 'magnesium', 'silicon', 'sulphur', 'calcium', 'alpha', 'parentid', 'partitionid', 'dform', 'id', 'log10_NH'])

The input data must be formatted as a dictionary of equal-length arrays. The dictionary must have the following entries:
- key `pos3`: particle position coordinates in $kpc$ (shape Nx3)
- key `vel3`: particle velocity coordinates in $km.s^{-1}$ (shape Nx3)
- key `mass`: particle stellar mass in solar masses
- key `age`: particle log10 stellar age in years
- key `feh`: particle stellar metallicity \[Fe/H\] in dex relative to solar

Additionally, the following entries can optionally be added:
- key `parentid`: index to give to the parent particle
- key `id`: additional index to classify the parent particle
- key `log10_NH`: log10 hydrogen column densities between Observer position and particle in $cm^{-2}$ - must be provided to estimate extinctions
- key `dform`: particle formation distance
- keys `helium`, `carbon`, `nitrogen`, `oxygen`, `neon`, `magnesium`, `silicon`, `sulphur`, `calcium`: particle various chemical abundances \[X/H\]
- key `alpha`: particle alpha chemical abundances \[Mg/Fe\]

Ananke will compute the phase space densities that are used to determine particle smoothing lengths, but one can include pre-computed densities with the following entries:
- key `rho_pos`: particle density in position space in $kpc^{-3}$
- key `rho_vel`: particle density in velocity space in $km^{-3}.s^{3}$

We can define here some parameters for Ananke such as
 - the observer position `observer`
 - the shell of particles to mask `rshell`
 - the sampling factor `fsample`
 - the photometric system of choise `photo_sys`
 - the CMD `cmd_magnames` and its box limits `cmd_box`

In [3]:
D = 200 # *units.kpc

observer = np.nan*np.ones(3)
while not np.linalg.norm(observer)<1:
    observer = 2*np.random.rand(3)-1

observer *= D/np.linalg.norm(observer)

rshell = [0, 2*D]

fsample = 0.01

photo_sys = 'padova/GAIA__DR2'

cmd_magnames = {'magnitude': 'G',
                'color_minuend': 'Gbp',
                'color_subtrahend': 'Grp'}

cmd_box = {
           'abs_mag_lim_lo': -1000,
           'abs_mag_lim_hi': 1000,
        #    'app_mag_lim_lo' : -1000,
           'app_mag_lim_hi': 30,
        #    'color_lim_lo' : -1000,
        #    'color_lim_hi' : 1000
           }

For more details regarding these parameters and more, you may consult the docstring associated to the class `Ananke`:

In [4]:
help(an.Ananke)

Help on class Ananke in module ananke.Ananke:

class Ananke(builtins.object)
 |  Ananke(particles: Dict[str, numpy.ndarray[Any, numpy.dtype[+_ScalarType_co]]], name: str, ngb: int = 64, d_params: Dict[str, Any] = {}, e_params: Dict[str, Any] = {}, err_params: Dict[str, Any] = {}, **kwargs: Dict[str, Any]) -> None
 |  
 |  Represents a single ananke pipeline.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, particles: Dict[str, numpy.ndarray[Any, numpy.dtype[+_ScalarType_co]]], name: str, ngb: int = 64, d_params: Dict[str, Any] = {}, e_params: Dict[str, Any] = {}, err_params: Dict[str, Any] = {}, **kwargs: Dict[str, Any]) -> None
 |      Parameters
 |      ----------
 |      particles : dict
 |          A dictionary of same-length arrays representing particles
 |          data of a stellar population - see notes for formatting
 |      
 |      name : str
 |          Name for the pipeline
 |      
 |      ngb : int
 |          Number of neighbours to use in kernel density estimatio

Preparing now the pipeline with GAIA photometry

In [5]:
name = 'sim'
ananke = an.Ananke(p, name, fsample=fsample,
                   observer=observer, rshell=rshell,
                   photo_sys=photo_sys, cmd_magnames=cmd_magnames,
                   **cmd_box)

/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/utils.py:23: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._record_of_all_used_keys = set()
/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/utils.py:23: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._record_of_all_used_keys = set()


The method `run` runs the pipeline:

In [6]:
survey = ananke.run()

Dimensions = 3
Normalization constant of Kernel type 3: 0.596831
Reading ASCII format file: to_enbid 
Read 300000 records
Allocated 3.8147 MByte for particle storage.
Type = 1 Particles = 100000
Total particles = 100000
Allocated 22.1252 MByte for Binary-Tree and Entropy bins.

Scaling Co-ordinates as x[i]=x[i]/h[i] with h[i]->
1 1 1 

Starting to Build Tree .......
Particle Type = 1 First node number = 1
                  Last node number  = 199999
Total number of nodes = 199999
Treebuild time = 0.102349 s 

Density Calculation. Smoothing .....
Evaluated =   1 % Time Left = 0.742295 s of 0.749800 s Par no = 96805 Density = 4.540411e-02 
Evaluated =   2 % Time Left = 0.713237 s of 0.727800 s Par no = 55454 Density = 4.964532e-03 
Evaluated =   3 % Time Left = 0.761280 s of 0.784833 s Par no = 27717 Density = 1.827217e-01 
Evaluated =   4 % Time Left = 0.751768 s of 0.783100 s Par no = 42234 Density = 2.712376e-02 
Evaluated =   5 % Time Left = 0.734437 s of 0.773100 s Par no = 99785 De

/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/utils.py:23: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  self._record_of_all_used_keys = set()
/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/_default_error_model.py:37: RuntimeWarning: overflow encountered in power
  grvs = rpmag + 132.32 - 377.28*ggrp + 402.32*ggrp**2 - 190.97*ggrp**3 + 34.026*ggrp**4
/home/athob/miniconda3/envs/main/lib/python3.10/site-packages/ananke/_default_error_model.py:37: RuntimeWarning: overflow encountered in multiply
  grvs = rpmag + 132.32 - 377.28*ggrp + 402.32*ggrp**2 - 190.97*ggrp**3 + 34.026*ggrp**4


Exported the following quantities to /home/athob/Software/repos/py-ananke/jupyter/survey.sim.0.h5
['pi_Err', 'mura_Sig', 'mudec_Sig', 'dec_Sig', 'vr_Sig', 'vr_Err', 'ra_Sig', 'pi_Sig', 'ra_Err', 'gaia__dr2_g_Err', 'mura_Err', 'gaia__dr2_grp_Sig', 'mudec_Err', 'gaia__dr2_grp_Err', 'dec_Err', 'gaia__dr2_gbp_Err', 'gaia__dr2_g_Sig', 'gaia__dr2_gbp_Sig']
Overwritten the following quantities to /home/athob/Software/repos/py-ananke/jupyter/survey.sim.0.h5
('gaia__dr2_grp', 'mura', 'gaia__dr2_gbp', 'mudec', 'pi', 'ra', 'vr', 'gaia__dr2_g', 'dec')
Running convert_icrs_to_galactic post-processing pipeline
Overwritten the following quantities to /home/athob/Software/repos/py-ananke/jupyter/survey.sim.0.h5
('glon', 'glat', 'mul', 'mub')


The output is saved as a `vaex` dataframe, with its columns organized in alphabetical order. These notably include:
- key `A_0` for the reference extinction which extinction coefficients are based on (at $\lambda = 550 \, nm$ in the case of Gaia DR2)
- key `A_{filter_name}` for the extinction value in each filter designated by `filter_name` (in this case, where `filter_name` is one of the 3 Gaia bands `gaia__dr2_gbp`, `gaia__dr2_grp` \& `gaia__dr2_g`)
- key `E(B-V)` for the reddening index
- key `age` for the log10 stellar age in years
- key `alpha`, `calcium`, `carbon`, `helium`, `magnesium`, `neon`, `nitrogen`, `oxygen`, `silicon`, `sulphur` for the various chemical abundances as given as input
- key `dec`, `ra` for the astrometric declination and right ascension celestial coordinates in degrees
- key `dform` for the formation distance as given as input
- key `dmod` for the distance modulus
- key `feh` for the stellar metallicity \[Fe/H\] in dex relative to solar
- key `glat`, `glon` for the astrometric galactic latitude and longitude celestrial coordinates in degrees
- key `grav` for the log10 surface gravity in CGS units
- key `log10_NH` for the log10 hydrogen column density between Observer position and star in $cm^{-2}$
- key `lum` for the stellar luminosity in solar luminosities
- key `mact`, `mtip`, `smass` for respectively the current stellar mass, the mass of that same star at tip of giant branch for its given age \& metallicity and its stellar mass on zero-age main sequence, all in solar masses
- key `mub`, `mudec`, `mul`, `mura` for the astrometric proper motions, respectively in the direction of the galactic latitude, declination, galactic longitude and right ascension, all in milliarcseconds per year
- key `parentid` for the parent particle index as given as input
- key `partid` for the flag that identifies stars that are *not* central relatively to their parent particle
- key `pi` for the star parallax in milliarcseconds
- key `px`, `py`, `pz` for the star position cartesian coordinates in $kpc$ relative to the Observer's position
- key `rad` for the star distance to the Observer in $kpc$
- key `teff` for the star effective temperature in Kelvin
- key `vr` for the star astrometric radial velocity in $km.s^{-1}$
- key `vx`, `vy`, `vz` for the star velocity cartesian coordinates in $km.s^{-1}$ relative to the Observer's velocity

Additionally, astrometric and photometric quantities `X` all have associated columns identified as:
- key `X_Sig` for the standard error on the quantity `X`
- key `X_Err` for the actual drawn gaussian error on the quantity `X`

In [7]:
survey

#,A_0,A_gaia__dr2_g,A_gaia__dr2_gbp,A_gaia__dr2_grp,E(B-V),age,alpha,calcium,carbon,dec,dec_Err,dec_Sig,dform,dmod,feh,gaia__dr2_g,gaia__dr2_g_Err,gaia__dr2_g_Intrinsic,gaia__dr2_g_Sig,gaia__dr2_gbp,gaia__dr2_gbp_Err,gaia__dr2_gbp_Intrinsic,gaia__dr2_gbp_Sig,gaia__dr2_grp,gaia__dr2_grp_Err,gaia__dr2_grp_Intrinsic,gaia__dr2_grp_Sig,glat,glon,grav,helium,log10_NH,lum,mact,magnesium,mtip,mub,mudec,mudec_Err,mudec_Sig,mul,mura,mura_Err,mura_Sig,neon,nitrogen,oxygen,parentid,partid,partitionid,pi,pi_Err,pi_Sig,px,py,pz,ra,ra_Err,ra_Sig,rad,satid,silicon,smass,sulphur,teff,vr,vr_Err,vr_Sig,vx,vy,vz
0,0.7093970129666607,0.5278885380814685,0.7125676091560069,0.40701678341228237,0.22883774611827765,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,nan,nan,0.0,20.87123488088669,-0.8867433071136475,nan,nan,7.6075797,nan,nan,nan,8.22037,nan,nan,nan,6.875088,nan,nan,nan,4.7529893,-0.7315406799316406,21.757467670308863,0.08502915,0.52380776,-0.5486690402030945,0.886202,nan,nan,nan,nan,nan,nan,nan,nan,-0.7326117157936096,-0.652482807636261,-0.6697278618812561,0,0,0,nan,nan,nan,-142.0352559796068,46.20841677042607,0.6925918281909844,nan,nan,nan,149.3643578805275,0,-0.7563512325286865,0.5240775107307198,-1.0924330949783325,4399.992,nan,nan,nan,67.38685760498046,-300.3354850769043,55.13861343383789
1,0.3913526151493549,0.29144945079467655,0.3932110342103401,0.22456925568778976,0.12624277908043705,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,nan,nan,0.0,20.87527071994942,-0.8867433071136475,nan,nan,7.8151436,nan,nan,nan,8.465081,nan,nan,nan,7.0564466,nan,nan,nan,4.767861,-0.7315406799316406,21.499146555104666,0.07226164,0.507031,-0.5486690402030945,0.886202,nan,nan,nan,nan,nan,nan,nan,nan,-0.7326117157936096,-0.652482807636261,-0.6697278618812561,0,1,0,nan,nan,nan,-142.26086176292293,46.417971376138425,-0.11534282804241658,nan,nan,nan,149.6422205133747,0,-0.7563512325286865,0.5073366488986605,-1.0924330949783325,4295.465,nan,nan,nan,67.7484200428451,-298.29957314526564,52.69162146715175
2,7.392986027528272,4.395135897711333,6.739848191949818,3.857969371581923,2.3848342024284745,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,nan,nan,0.0,20.888157433594138,-0.8867433071136475,nan,nan,8.512117,nan,nan,nan,9.277008,nan,nan,nan,7.679845,nan,nan,nan,4.8335814,-0.7315406799316406,22.775398200235458,0.040845014,0.44261116,-0.5486690402030945,0.886202,nan,nan,nan,nan,nan,nan,nan,nan,-0.7326117157936096,-0.652482807636261,-0.6697278618812561,0,1,0,nan,nan,nan,-142.92704618284074,47.20842184009032,1.8396875936103667,nan,nan,nan,150.53292023217344,0,-0.7563512325286865,0.4436048636953867,-1.0924330949783325,4001.0637,nan,nan,nan,68.03093236871236,-301.4758806839234,56.09200627357448
3,3.6129888513390416,2.2818297625233486,3.4033181539962825,1.9506143576453838,1.1654802746254973,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,nan,nan,0.0,20.867217518655195,-0.8867433071136475,nan,nan,9.037168,nan,nan,nan,9.901029,nan,nan,nan,8.152237,nan,nan,nan,4.8903975,-0.7315406799316406,22.46444493629801,0.027340515,0.39295405,-0.5486690402030945,0.886202,nan,nan,nan,nan,nan,nan,nan,nan,-0.7326117157936096,-0.652482807636261,-0.6697278618812561,0,1,0,nan,nan,nan,-141.29345876416804,47.56378569643568,1.0770165239203249,nan,nan,nan,149.08827976709273,0,-0.7563512325286865,0.39350367992526536,-1.0924330949783325,3854.4998,nan,nan,nan,69.12724120992327,-301.73920387771557,55.1029961513569
4,0.7014795175813543,0.5867886042288581,0.7483665104609659,0.4149519539856739,0.22628371534882397,10.016325950622559,0.3380742371082306,-1.1830222606658936,-0.39430123567581177,nan,nan,nan,0.0,20.86868071849355,-0.8867433071136475,nan,nan,4.9383516,nan,nan,nan,5.201803,nan,nan,nan,4.5038996,nan,nan,nan,4.4819903,-0.7315406799316406,21.752593309473333,0.8118422,0.7637918,-0.5486690402030945,0.886202,nan,nan,nan,nan,nan,nan,nan,nan,-0.7326117157936096,

Please refer to [`vaex`'s documentation](https://vaex.io/docs/tutorial.html) for further help on how to use `vaex` dataframes: the following line for example isolate only the rows with non-NaN photometry.

In [8]:
survey[~survey.gaia__dr2_g.isna()]

#,A_0,A_gaia__dr2_g,A_gaia__dr2_gbp,A_gaia__dr2_grp,E(B-V),age,alpha,calcium,carbon,dec,dec_Err,dec_Sig,dform,dmod,feh,gaia__dr2_g,gaia__dr2_g_Err,gaia__dr2_g_Intrinsic,gaia__dr2_g_Sig,gaia__dr2_gbp,gaia__dr2_gbp_Err,gaia__dr2_gbp_Intrinsic,gaia__dr2_gbp_Sig,gaia__dr2_grp,gaia__dr2_grp_Err,gaia__dr2_grp_Intrinsic,gaia__dr2_grp_Sig,glat,glon,grav,helium,log10_NH,lum,mact,magnesium,mtip,mub,mudec,mudec_Err,mudec_Sig,mul,mura,mura_Err,mura_Sig,neon,nitrogen,oxygen,parentid,partid,partitionid,pi,pi_Err,pi_Sig,px,py,pz,ra,ra_Err,ra_Sig,rad,satid,silicon,smass,sulphur,teff,vr,vr_Err,vr_Sig,vx,vy,vz
0,0.05681650709265598,0.04671342391859208,0.05985558882052393,0.03353577934926581,0.018327905513759995,8.97172737121582,-0.6125200986862183,-0.12460324913263321,-0.36052706837654114,41.517887,-4.3651322762456503e-07,3.7339328e-07,0.0,20.938860082853388,-0.5348672270774841,20.610794,-0.013108958690914031,-0.36166972,0.014190837,20.934616,-0.09105875752639689,0.026959075,0.32711947,20.067835,0.010476073807142064,-0.9150356,0.32711947,-6.450095,159.08374,2.5680106,-0.6886086463928223,20.661052845931597,108.46986,2.1231031,-1.1473873853683472,1.9929105,1.9060571825344517,-0.4903980049570771,-0.15208820165614972,2.1212947,2.637362501364926,3.216868859449929,3.107170100412025,2.1212947,-0.44875627756118774,-1.4166113138198853,-0.40134233236312866,138,1,0,0.004700138418892006,-0.0017896118250197454,1.3442158,-143.02405261900319,54.66209816691588,-17.310021738871935,64.385994,7.738775790567222e-07,3.7339328e-07,154.089134776509,0,-0.4361392557621002,2.123614686399606,-1.2804145812988281,5266.821,nan,nan,nan,-13.980939858346646,-241.322783466562,-112.49946999755836
1,0.30908957284836036,0.2261170869376632,0.30842259877311534,0.176280263594307,0.09970631382205172,10.069992065429688,0.5896989703178406,-0.9664415121078491,-0.9518471360206604,37.240967,-2.2831375342149955e-07,2.49317e-07,0.0,21.769605166840982,-0.801622748374939,20.222652,-0.013217448221518976,-1.7598503,0.011413363,21.307426,0.2680249694291372,-1.0386255,0.24083452,19.243824,-0.13966843269274323,-2.5623941,0.24083452,-0.91259795,169.44246,1.0347817,-0.32748517394065857,21.396662669202907,558.4033,0.8450456,-0.21192379295825958,0.86665446,-3.1047273926896426,-2.7124202720419555,-2.542487626107496,1.4928615,1.1050811506348799,-1.871713888922517,-2.04588855812214,1.4928615,-0.3413775563240051,-0.13905183970928192,-0.8814153671264648,212,1,0,-1.1660510794203147,-1.1704777679639904,0.89754117,-222.0501127897472,41.38527019482137,-3.5979765713472083,78.53204,4.2180988179474246e-07,2.49317e-07,225.90249802613027,0,-0.7138696312904358,0.8677262727803007,-1.077961802482605,4132.9897,nan,nan,nan,-64.76429401098572,-249.0524176755581,44.549935511279955
2,0.040653863770659916,0.03188868360172066,0.04180280722924421,0.023700606699203153,0.013114149603438681,9.613767623901367,-0.1432933509349823,-1.0196176767349243,-0.6505561470985413,27.468527,9.93141102529504e-07,5.270746e-07,0.0,21.241844522741097,-0.5965235829353333,20.921713,-0.029182178287689993,-0.3228382,0.01705969,21.876608,0.38537958340209455,0.20757976,0.4235812,21.28318,1.0115375512029108,-0.99390167,0.4235812,-8.830861,175.76826,2.0586061,-0.7592963576316833,20.51568014237018,118.69824,1.196,-0.7398169040679932,1.1982795,-1.6438476299687612,-2.044957683093608,-1.7305400665725954,2.8642974,1.322617557411089,-0.519326655124296,-0.5989357582041593,2.8642974,-1.3291157484054565,-0.7577531337738037,-0.45060980319976807,252,1,0,1.5601152190393899,1.554470646021109,1.8974684,-174.58395850876306,12.917886544408068,-27.197472653752673,75.42295,-1.738821925276305e-07,5.270746e-07,177.1613188032741,0,-0.6941993832588196,1.2002945213596574,-0.5894879698753357,4637.9014,nan,nan,nan,31.915626870159116,-254.29642034774957,-98.61775218664684
3,0.7877962999055037,0.5726781831841866,0.7836102802990144,0.4485485017244748,0.25412783867919475,9.8711519241333,1.162867546081543,-0.7159809470176697,-0.6287623643875122,34.62386,6.854075293976649e-08,7.639342e